In [ ]:
# chrome.exe -remote-debugging-port=9014 --user-data-dir="C:\Users\deepa\Desktop\LinkedinExtract\ChromeData"
import os
os.system('cmd /k chrome.exe -remote-debugging-port=9014 --user-data-dir="C:\Users\deepa\Desktop\LinkedinExtract\ChromeData"')

In [ ]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd


In [ ]:
remote_debugging_port = 9014


options = Options()
options.headless = False
options.add_experimental_option('debuggerAddress', 'localhost:9014')

url="https://www.linkedin.com/jobs/search/?currentJobId=3628684746&distance=25&f_TPR=r86400&geoId=102713980&keywords=software&refresh=true&start=25"
# url="https://www.linkedin.com/login"

driver = webdriver.Chrome("chromedriver.exe", options=options)
driver.execute_cdp_cmd("Network.enable", {})

driver.get(url)

# sleep(2)
# driver.find_element_by_name('session_key').send_keys("USERNAME")
# sleep(1)
# driver.find_element_by_name('session_password').send_keys("PASSWORD")

element = WebDriverWait(driver, 20).until(
EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/header/nav/div/a[2]')))

element.click();

wait = WebDriverWait(driver, 10)
wait.until(EC.element_to_be_clickable((By.NAME, "session_key"))).send_keys('USERNAME')
wait = WebDriverWait(driver, 10)
wait.until(EC.element_to_be_clickable((By.NAME, "session_password"))).send_keys('PASSOWRD')

element = WebDriverWait(driver, 20).until(
EC.element_to_be_clickable((By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')))

element.click();


# Get all network request data
response_bodies = []
for entry in driver.execute_cdp_cmd("Network.getAllResponseBody", {}).get("base64Encoded", []):
    response_bodies.append(entry.get("body", ""))

# Process the response bodies (JSON data)
for response_body in response_bodies:
    print(response_body)
# open_tabs = driver.window_handles

# # Print the URLs of the open tabs
# for tab in open_tabs:
#     driver.switch_to.window(tab)
#     print(driver.current_url)

In [ ]:
# name 
# company 
# location 
# nature 
# time_posted 
# position_level 
# job_description 
# skills_identified
# job_link

In [ ]:
def extract_text(element):
    text = element.text.strip()
    for sub_element in element.find_elements_by_xpath(".//*"):
        text += " " + sub_element.text.strip()
    return text


In [ ]:
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)

finallist=[]
for i in range(1,26):
    
    jobtag=f'/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[1]/div/ul/li[{i}]'
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, jobtag)))

    element.click();
    
    # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "jobs-company__box")))


    job_data=dict()
    
    element = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div[1]/a/h2')))
    job_data['position']=element.text
    
    element = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'jobs-unified-top-card__company-name')))
    job_data['company']=extract_text(element)

    element = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div[2]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div[2]/span[1]/span[2]')))
    job_data['location']=extract_text(element)
    
    element = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(
        # EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[3]/div[4]/div/div/main/div/div[2]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div[2]/span[1]/span[3]')))
        EC.presence_of_element_located((By.CLASS_NAME, 'jobs-unified-top-card__workplace-type')))

    job_data['nature']=extract_text(element)

    element = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div[2]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div[2]/span[2]/span[1]')))
    job_data['time_posted']=extract_text(element)
    
    element = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div[2]/div/div[2]/div[1]/div/div[1]/div/div[1]/div[1]/div[3]/ul/li[1]/span')))
    job_data['position_level']=extract_text(element)
    
    job_data['description']=None
    while(job_data['description'] is None):
        try:
            element = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="job-details"]')))
            job_data['description']=extract_text(element)
        except:
            pass
        
    # job_data['skills_identified']=None
    # while(job_data['skills_identified'] is None):
    #     try:
    #         element = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(
    #             EC.presence_of_element_located((By.CLASS_NAME, 'pt5')))
    #         job_data['skills_identified']=extract_text(element)
    #     except:
    #         pass

    job_data['job_link']=driver.current_url
    
    finallist.append(job_data)
    


   
    

In [ ]:
df = pd.DataFrame(finallist)
df.to_csv("asd.csv")

In [ ]:
from selenium import webdriver
from browsermobproxy import Server

# Start the BrowserMob Proxy server
server = Server("path/to/browsermob-proxy")
server.start()
proxy = server.create_proxy()

# Configure Chrome WebDriver to use the proxy
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))

# Start Chrome WebDriver with the configured options
driver = webdriver.Chrome(options=chrome_options)

# Enable network capture
proxy.new_har("capture")

# Open the web page
driver.get("https://www.example.com")

# Perform actions on the page that trigger network requests

# Get the captured network traffic
har = proxy.har

# Find the JSON response(s) and extract the data
json_responses = []
for entry in har["log"]["entries"]:
    response = entry["response"]
    content_type = response["content"]["mimeType"]
    if content_type == "application/json":
        json_data = response["content"]["text"]
        json_responses.append(json_data)

# Print the JSON responses
for json_data in json_responses:
    print(json_data)

# Close the browser and proxy server
driver.quit()
server.stop()


In [28]:
import requests

# Set up session with cookies and CSRF token
session = requests.Session()
cookies = {'cookie_name': 'bcookie="v=2&853d89cf-2efd-4105-852a-ce343ee05781"; bscookie="v=1&20221206065043a639f48c-9406-405b-875f-f5dd13258189AQGNGj5WI_RcJSXLHrhd2QD8jOGV16po"; G_ENABLED_IDPS=google; aam_uuid=16662723610550796321673254091152595947; JSESSIONID="ajax:7665625736133769427"; timezone=Asia/Calcutta; li_sugr=2eb504a4-954f-4f3e-96b4-417d2adabbd2; li_theme=dark; li_theme_set=user; _gcl_au=1.1.2097665459.1670577636; mbox=session#8c98c551e6ef40ebaa1e323d594ffef1#1678212699|PC#8c98c551e6ef40ebaa1e323d594ffef1.31_0#1693762839; _guid=c0fb0f13-b8be-4f6e-a6f7-0d751548e413; gpv_pn=www.linkedin.com%2Fpremium%2Fsurvey%2F; s_ips=837; s_tslv=1683890015601; s_tp=2794; liap=true; AnalyticsSyncHistory=AQJqi0pSmBuagQAAAYiXP5yyAbTBWWetv2Ig_hhMldqgWMeHXRKp65utviwAe-qjwLUVFtpl_K4cYRSUZvmAPA; lms_ads=AQEOPdh67m0x_gAAAYiXP52iUNOvNIrkQUusmn2gEL0vtV2woYFWKHOgmSLYQUAwEExWTQQr4GUWJcC8BTyQAvnPosYXgKbn; lms_analytics=AQEOPdh67m0x_gAAAYiXP52iUNOvNIrkQUusmn2gEL0vtV2woYFWKHOgmSLYQUAwEExWTQQr4GUWJcC8BTyQAvnPosYXgKbn; li_at=AQEDASiK4OEF3UsAAAABiJsY3-gAAAGIvyVj6FYAwmCSTW0Ga-8BtNJIZR9ppknAr2VeW9oaAB1SwxJusj5LL3_OEP6n1eTyta81pfRrMXMy2w8qtlnbcwo7MQKjLvD_MoKt4W-5NcesfUFmhvLT3oyk; ln_or=eyI0MzQ2MTM3IjoiZCJ9; lang=v=2&lang=en-us; AMCVS_14215E3D5995C57C0A495C55%40AdobeOrg=1; AMCV_14215E3D5995C57C0A495C55%40AdobeOrg=-637568504%7CMCIDTS%7C19519%7CMCMID%7C17161253032002680591727051875148916768%7CMCAAMLH-1686990198%7C12%7CMCAAMB-1686990198%7CRKhpRz8krg2tLO6pguXWp5olkAcUniQYPHaMWWgdJ3xzPWQmdj0y%7CMCOPTOUT-1686392598s%7CNONE%7CvVersion%7C5.1.1%7CMCCIDH%7C856016164; UserMatchHistory=AQKKHcMy63pPVgAAAYikk9hqWyWjF4ohF6WQUJKgWElxNhmHrcqaR6T0sw1PVuV9c3aXr9H7lQ4-19mi-gzovrkDOI0IrI98l5T4mvoo_fQs8aVvWc9WQGmCpUJBjJkWIkA-K_0AOtvLzlSUTjJ4PcLUwTKMeg_ap2h-18ZgiQbtZfWjAnASdRtuDX5LtiRGObjjDgABsj0CeNQTV868fhPnE_bqeJY_nVQNagOSf9RptabRQ1lK6cPMm1RHI7ZEVXZ3UFmSPzO8owNuoB3aDdsOiOMAW98nbctd-PjLKc0aJYvdSb08iuVtQGcOGM6mqnYevCTe__koJOhMiRoB2p5OW6Sblw0; lidc="b=OGST09:s=O:r=O:a=O:p=O:g=2531:u=1:x=1:i=1686388333:t=1686474733:v=2:sig=AQGK6ZJM4qhYwFsIiv0IO9On96AlKcvr"'}  # Replace with your actual cookie
csrf_token = 'ajax:7665625736133769427'  # Replace with your actual CSRF token

# Set the CSRF token in the headers
headers = {'Csrf-Token': csrf_token}

# Update the session cookies and headers
session.cookies.update(cookies)
session.headers.update(headers)

# Make a GET request to the desired URL
url = "https://www.linkedin.com/voyager/api/jobs/jobPostings/3609692568"
response = session.get(url)

print(response)
# Retrieve and process the JSON response
# json_data = response.json()
# # ... process the JSON data as needed

# # Close the session
# session.close()


<Response [200]>


In [29]:
response

<Response [200]>

In [1]:
type(response.json())

NameError: name 'response' is not defined

In [2]:
import spacy

def extract_technical_skills(job_description):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(job_description)
    
    technical_skills = []
    for ent in doc.ents:
        if ent.label_ == 'SKILL':
            technical_skills.append(ent.text)
    
    return technical_skills

In [3]:
job_description = "We are seeking a candidate with experience in Python, Java, and SQL. Familiarity with machine learning libraries such as TensorFlow or PyTorch is a plus."

extracted_skills = extract_technical_skills(job_description)
print(extracted_skills)


[]
